## Bigrams - Log Likelihood Ratio

First suggested by Ted Dunning, see [blog post](http://tdunning.blogspot.com/2008/03/surprise-and-coincidence.html).

Part of family of algorithms that finds collocations on basis of probability, i.e., a bigram is a keyword if the probability of the two words co-occurring is high compared with the probability of them appearing independently. So:

$$P(B|A) \gg P(A)P(B)$$

Other algorithms in this family are (see [Foundations of Statistical Natural Language Processing, Chapter 5](https://nlp.stanford.edu/fsnlp/promo/colloc.pdf) by Manning and Schutze.

* t-test
* Chi-squared test
* likelihood ratios
* mutual information

With Dunnings LLR method, we want to find the likelihood $log \lambda$ of the likelihoods of the following two hypothesis.

$$H1: P(w_2|w_1) = p = P(w_2|-w_1)$$

$$H2: P(w_2|w_1) = p1 \ne p2 = P(w_2|-w_1)$$

Here H1 is the independence hypothesis and H2 is the dependence hypothesis.

We can compute the probabilities $p$, $p_1$ and $p_2$ in terms of various counts as follows:

$$p = \frac{c_2}{N}$$

$$p_1 = \frac{c_{12}}{c_1}$$

$$p_2 = \frac{c_2 - c_{12}}{N - c_1}$$

Assuming that words follow a binomial distribution, the log likelihood ratio is given by the following formula.

$$log \lambda = log \frac{L(H1)}{L(H2)} = log \frac{L(c_{12}, c_1, p) L(c_2 - c_{12}, N-c_1, p)}{L(c_{12}, c_1, p_1) L(c_2 - c_{12}, N-c_1, p_2)}$$

where:

$$L(k, n, x) = x^k (1 - x)^{n-k}$$

We will calculate the LLR for the bigrams we discovered in the `04-bigrams` notebook and saved in `candidate_bigrams.tsv`.

In [1]:
import math
import operator
import os
import sqlite3

In [2]:
DATA_DIR = "../data"

CANDIDATE_BIGRAMS = os.path.join(DATA_DIR, "candidate_bigrams.tsv")
WORDCOUNTS_DB = os.path.join(DATA_DIR, "wordcounts.db")

BIGRAM_KEYWORDS = os.path.join(DATA_DIR, "bigram_keywords.tsv")

In [3]:
conn = sqlite3.connect(WORDCOUNTS_DB)

In [4]:
def count_num_rows(conn, table_name):
    cur = conn.cursor()
    cur.execute("select count(*) as freq from {:s}".format(table_name))
    rows = cur.fetchone()
    return int(rows[0])
    cur.close()

N = count_num_rows(conn, "wordcounts")
print(N)

46889932


In [5]:
def unigram_count(conn, unigram):
    cur = conn.cursor()
    cur.execute("select count(*) as freq from wordcounts where word = ?", [unigram])
    rows = cur.fetchone()
    return int(rows[0])
    cur.close()
    

def log_likelihood(k, n, x):
    # L(k, n, x) = x ** k * (1-x) ** (n-k)
    # multiple if conditions to escape math.log ValueError
    if x == 0:
        return (n - k) * math.log(1 - x)
    elif x == 1:
        return k * math.log(x)
    else:
        return k * math.log(x) + (n - k) * math.log(1 - x)


def log_likelihood_ratio(c1, c2, c12, p, p1, p2, N):
    return (log_likelihood(c12, c1, p) + log_likelihood(c2 - c12, N - c1, p) -
        log_likelihood(c12, c1, p1) - log_likelihood(c2 - c12, N - c1, p2))

In [6]:
llrs = []
fbig = open(CANDIDATE_BIGRAMS, "r")
for line in fbig:
    word_1, word_2, c12 = line.strip().split("\t")
    c12 = int(c12)
    c1 = unigram_count(conn, word_1)
    c2 = unigram_count(conn, word_2)
    p = c2 / N
    p1 = c12 / c1
    p2 = (c2 - c12) / (N - c1)
    llr = log_likelihood_ratio(c1, c2, c12, p, p1, p2, N)
#     print(word_1, word_2, llr)
    llrs.append((word_1, word_2, llr))

fbig.close()

print("{:d} candidate keywords predicted".format(len(llrs)))

2280 candidate keywords predicted


### Ordering bigrams by LLR

LLR is ratio of likelihood of independence by likelihood of dependence, so more probable bigrams means lower values of LLR, hence ordering should be by LLR ascending.

In [7]:
sorted_llrs = sorted(llrs, key=operator.itemgetter(2))
fbk = open(BIGRAM_KEYWORDS, "w")
i = 0
for word_1, word_2, llr in sorted_llrs:
    if i <= 10:
        print(word_1, word_2, llr)
    fbk.write("{:s}\t{:s}\t{:.3f}\n".format(word_1, word_2, llr))
    i += 1
    
fbk.close()

et al -151518.48289648563
machine learning -74404.30454820918
neural networks -56336.203218498355
processing systems -54158.87871827083
information processing -50136.83927513713
international conference -46681.889256491806
neural information -37865.34878083441
neural network -32324.2155052608
arxiv preprint -31963.201131558242
mit press -29930.99386955142
monte carlo -29890.898612344055
